In [ ]:
import matplotlib.pyplot as plt
import tensorflow.keras as keras

%matplotlib inline
import sys

import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, Sequential

sys.path.append("..")

In [ ]:
img_rows, img_cols = 28, 28
(x_train, y_train_), (x_test, y_test_) = mnist.load_data()
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255.0
x_test /= 255.0
y_train = keras.utils.to_categorical(y_train_)
y_test = keras.utils.to_categorical(y_test_)

In [ ]:
model = Sequential()
model.add(Dense(500, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["acc"])

In [ ]:
model.fit(x_train, y_train, validation_split=0.2, batch_size=32, epochs=1, verbose=0)

In [ ]:
# model.save_weights('tuto_4_ongoing.hd5')

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
model.predict(x_train[:1]).argmax()

In [ ]:
from decomon.models import clone

In [ ]:
from decomon import get_adv_box

In [ ]:
eps = 0.1
X_train = x_train
X_train_min = X_train - eps
X_train_max = X_train + eps
Y_train = y_train
B_train = np.concatenate([X_train_min[:, None], X_train_max[:, None]], 1)

In [ ]:
back_bounds = Input((10, 10))

In [ ]:
decomon_nnet = clone(model, method="crown-ibp", back_bounds=[back_bounds], finetune=False)

In [ ]:
from decomon.metrics import build_crossentropy_model

In [ ]:
nnet = build_crossentropy_model(decomon_nnet)

In [ ]:
nnet.compile("adam", "categorical_crossentropy")

In [ ]:
C = np.diag([1] * 10)[None] - y_train[:, :, None]

In [ ]:
get_adv_box(decomon_nnet, B_train[:10, 0], B_train[:10, 1], Y_train[:10])

In [ ]:
nnet.fit([B_train, C], Y_train, validation_data=0.2, epochs=10)

In [ ]:
decomon_nnet_ = clone(model, method="crown-hybrid", finetune=False)

In [ ]:
get_adv_box(decomon_nnet_, B_train[:1, 0], B_train[:1, 1], Y_train[:1])

In [ ]:
decomon_nnet.input

In [ ]:
C = np.diag([1] * 10)[None] - y_train[:, :, None]

In [ ]:
_, u_, _, _, l_, _, _ = decomon_nnet.predict([B_train[:1], C[:1]])

In [ ]:
C = np.diag([1] * 10)[None]  # - y_train[:,None,:]
_, u, _, _, l, _, _ = decomon_nnet.predict([B_train[:1], C[:1]])

In [ ]:
u - l[0, 5]

In [ ]:
u_

In [ ]:
u

In [ ]:
l[0, 5]

In [ ]:
from decomon.metrics import build_radius_robust_model

In [ ]:
toto = build_radius_robust_model(decomon_nnet)

In [ ]:
toto.inputs

In [ ]:
eps = 0.015
X_train = x_train
X_train_min = X_train - eps
X_train_max = X_train + eps
Y_train = y_train
B_train = np.concatenate([X_train_min[:, None], X_train_max[:, None]], 1)

In [ ]:
toto.predict(B_train[:1])

In [ ]:
toto.predict(B_train[:1]).argmin()

In [ ]:
Y_train[:1].argmax()

In [ ]:
def loss(y_true, y_pred):
    return K.sum(y_true * y_pred, -1)

In [ ]:
nnet.compile("adam", loss)

In [ ]:
toto.fit(B_train[:10], Y_train[:10], epochs=10)

In [ ]:
decomon_crown = clone(model, method="crown", finetune=False)

In [ ]:
decomon_crown.inputs

In [ ]:
def build_graph(model, dim=(2, 784)):
    x = Input(shape=dim)
    return Model(inputs=[x], outputs=model.call(x))

In [ ]:
toto = build_graph(decomon_crown).summary()

In [ ]:
tf.keras.utils.plot_model(build_graph(decomon_crown), to_file="crown.png", expand_nested=True, show_shapes=True)

In [ ]:
from decomon import get_lower_box, get_upper_box

In [ ]:
decomon_crown.summary()

In [ ]:
decomon_crown.method

In [ ]:
f_ibp = get_upper_box(decomon_nnet, B_train[:100, 0], B_train[:100, 1], batch_size=10)

In [ ]:
g_ibp = get_lower_box(decomon_nnet, B_train[:100, 0], B_train[:100, 1], batch_size=10)

In [ ]:
f_crown = get_upper_box(decomon_crown, B_train[:100, 0], B_train[:100, 1], batch_size=10)
g_crown = get_lower_box(decomon_crown, B_train[:100, 0], B_train[:100, 1], batch_size=10)

In [ ]:
import time
start_time = time.process_time()
y_ibp = 

In [ ]:
# convert to training

In [ ]:
from decomon.metrics import build_crossentropy_model

In [ ]:
decomon_training = build_crossentropy_model(decomon_nnet)

In [ ]:
decomon_evaluate = build_crossentropy_model(decomon_crown)

In [ ]:
eps = 0.015
X_train = x_train
X_train_min = X_train - eps
X_train_max = X_train + eps
Y_train = y_train
B_train = np.concatenate([X_train_min[:, None], X_train_max[:, None]], 1)

In [ ]:
output_ibp = decomon_training.predict(B_train[:100])

In [ ]:
output_crown = decomon_evaluate.predict(B_train[:100], batch_size=10)

In [ ]:
y_ibp = np.sum((output_ibp * Y_train[:100]), -1)

In [ ]:
y_crown = np.sum((output_crown * Y_train[:100]), -1)

In [ ]:
np.allclose(output_ibp, output_crown)

In [ ]:
plt.scatter(y_crown, y_ibp)

In [ ]:
decomon_training.compile("adam", loss, metrics=["acc"])

In [ ]:
def loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [ ]:
decomon_training.compile("adam", loss, metrics=["acc"])

In [ ]:
decomon_training.fit(B_train, Y_train, validation_split=0.2, batch_size=100, epochs=1)

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
from decomon import get_adv_box, get_adv_loss, get_model
from decomon.models import clone

In [ ]:
convex_domain = {"name": "ball", "p": np.inf, "eps": max(0, 0.015)}

In [ ]:
crown_ibp_ball = clone(model, method="crown-hybrid", convex_domain=convex_domain, finetune=False)

In [ ]:
crown_ibp_box = clone(model, method="crown-hybrid", finetune=False)

In [ ]:
from decomon import get_adv_box, get_adv_noise

In [ ]:
# build robustness bounds
eps = 0.015
X_train = x_train
X_train_min = X_train - eps
X_train_max = X_train + eps
Y_train = y_train
B_train = np.concatenate([X_train_min[:, None], X_train_max[:, None]], 1)

X_test = x_test
X_test_min = X_test - eps
X_test_max = X_test + eps
Y_test = y_test
B_test = np.concatenate([X_test_min[:, None], X_test_max[:, None]], 1)

In [ ]:
adv_ball = get_adv_noise(crown_ibp_ball, X_train[:10], Y_train[:10], eps=0.015, p=np.inf)

In [ ]:
adv_box = get_adv_box(crown_ibp_box, X_train_min[:10], X_train_max[:10], Y_train[:10])

In [ ]:
adv_ball

In [ ]:
adv_box

In [ ]:
def get_adv_noise(
    model,
    x,
    source_labels,
    eps=0,
    p=np.inf,
    target_labels=None,
    batch_size=-1,
    fast=True,
):

In [ ]:
adv_model = get_model(crown_ibp)

In [ ]:
adv_loss = get_adv_loss(crown_ibp, softmax=False)

In [ ]:
adv_model.compile("adam", adv_loss)

In [ ]:
# model.save_weights('tuto_4_ongoing.hd5')

In [ ]:
adv_model.evaluate(B_train[:1], Y_train[0:1])

In [ ]:
adv_model.fit(B_train[:4], Y_train[:4], batch_size=10, epochs=10)

In [ ]:
crown_hybrid = clone(model, method="crown-hybrid")

In [ ]:
adv_train = get_adv_box(crown_ibp, X_train_min[:10], X_train_max[:10], Y_train[:10])

In [ ]:
adv_train

In [ ]:
upper = get_adv_box(decomon_model, x_min, x_max, source_labels=source_label, n_sub_boxes=4)

In [ ]:
adv_model.evaluate(B, Y)